# imports

In [ ]:
from noises import *
from phantom import *

import torch
import torch.nn.functional as F

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as compare_psnr

# create folders

In [ ]:
# make directories
for key in NOISE_FUNCTIONS.keys():
    os.makedirs(f'./{key}', exist_ok=True)

In [ ]:
# make directories in the ground truth folder for each resolution
# 64, 128, 256, 512
for res in [64, 128, 256, 512]:
    os.makedirs(f'./ground_truth/{res}', exist_ok=True)


# generate ground truth and sample

In [ ]:
# now generate 50 phantoms for each resolution
# save the phantoms to the ground truth folder
for res in [6, 7, 8, 9]:
    for i in range(50):
        # res is 2**res
        # so a 64x64 image has a res of 6
        phantom = generate_phantom(res)
        # save the phantom
        # output is a np array
        np.save(f'./ground_truth/{2**res}/{i}.npy',phantom)

In [ ]:
# test plotting one of the phantoms

for res in [6, 7, 8, 9]:
    plt.figure()
    plt.imshow(np.load(f'./ground_truth/{2**res}/0.npy').squeeze(), cmap='gray')

# test noise

In [ ]:
# test applying each noise function to one of the phantoms
# the noise function works like this: add_selected_noise(img, noise_type='gaussian', **kwargs):

img = np.load(f'./ground_truth/64/0.npy').squeeze()
img_torch = phantom_to_torch(img)

for key in NOISE_FUNCTIONS.keys():
    plt.figure()
    noise_img = add_selected_noise(img_torch, noise_type=key)
    # include label of the noise type
    plt.title(key)
    plt.imshow(noise_img.squeeze(), cmap='gray', )

# note all noise is equal

In [ ]:

NOISE_FUNCTIONS = {
    'gaussian': {'function': add_gaussian_noise, 'param': 'noise_factor'},
    'salt_and_pepper': {'function': add_salt_and_pepper_noise, 'param': 'salt_prob'},  # Or 'pepper_prob'
    'speckle': {'function': add_speckle_noise, 'param': 'noise_factor'},
    'poisson': {'function': add_poisson_noise, 'param': None},
    'uniform': {'function': add_uniform_noise, 'param': 'noise_factor'},
    'exponential': {'function': add_exponential_noise, 'param': 'scale'},
    'rayleigh': {'function': add_rayleigh_noise, 'param': 'scale'},
    'erlang': {'function': add_erlang_noise, 'param': 'shape'},  # You can also add 'scale' depending on what you want to vary
    'brownian': {'function': add_brownian_noise, 'param': 'noise_factor'},
    'quantization': {'function': add_quantization_noise, 'param': 'levels'},
    'stripe': {'function': add_stripe_noise, 'param': 'noise_factor'},
    'multiplicative': {'function': add_multiplicative_noise, 'param': 'noise_factor'}
}


def psnr(input, target, max_val=1.):
    mse = F.mse_loss(input, target)
    return 20 * torch.log10(max_val / torch.sqrt(mse))

def find_noise_factor_for_psnr(target_psnr, img, noise_function, param_name, max_iter=1000, tol=1e-2, **kwargs):
    low, high = 0., 1.
    for i in range(max_iter):
        mid = (low + high) / 2
        kwargs[param_name] = mid
        noisy_img = noise_function(img, **kwargs)
        current_psnr = psnr(img, noisy_img)
        if (target_psnr - tol) <= current_psnr <= (target_psnr + tol):
            return mid
        if current_psnr < target_psnr:
            high = mid
        else:
            low = mid
    return mid


img = np.load(f'./ground_truth/64/0.npy').squeeze()
img_torch = phantom_to_torch(img)
target_psnr_value = 27.6 # Replace with your target PSNR value

for key, val in NOISE_FUNCTIONS.items():
    noise_func = val['function']
    param_name = val['param']
    if param_name:  # Skip noise functions that do not have a parameter to optimize
        optimal_noise_factor = find_noise_factor_for_psnr(target_psnr_value, img_torch, noise_func, param_name)
        # round the values to 5 decimal places
        print(f"Optimal {param_name} for {key} noise to achieve PSNR ~ {target_psnr_value}: {round(optimal_noise_factor, 5)}")




In [ ]:
# now do this for 1000 runs and average the results
# save the results to a csv file

# will need to adjust salt and pepper as that is the only one that has two parameters

img = np.load(f'./ground_truth/64/0.npy').squeeze()
img_torch = phantom_to_torch(img)
target_psnr_value = 15.7

num_runs = 1000  # Number of times to run the optimization for each noise type
optimal_factors_table = {}  # Initialize an empty dictionary to store the results

for key, val in NOISE_FUNCTIONS.items():
    noise_func = val['function']
    param_name = val['param']
    if param_name:  # Skip noise functions that do not have a parameter to optimize
        optimal_factors = []
        for i in range(num_runs):
            optimal_factor = find_noise_factor_for_psnr(target_psnr_value, img_torch, noise_func, param_name)
            optimal_factors.append(optimal_factor)

        avg_optimal_factor = np.mean(optimal_factors)
        # Round the values to 5 decimal places
        optimal_factors_table[key] = round(avg_optimal_factor, 5)

# Convert the dictionary to a Pandas DataFrame for better visual representation and ease of exporting
df = pd.DataFrame(list(optimal_factors_table.items()), columns=['Noise_Type', 'Avg_Optimal_Factor'])

# Print the DataFrame to view the results
print(df)

# Save the DataFrame to a CSV file
df.to_csv('optimal_noise_factors_matching20g.csv', index=False)

# and the 2d salt and pepper case

In [ ]:

def find_noise_factor_for_psnr_2D(target_psnr, img, noise_function, param_names, max_iter=1000, tol=1e-2, **kwargs):
    opt_params = {}
    for param1 in np.linspace(0, 1, max_iter):
        for param2 in np.linspace(0, 1, max_iter):
            kwargs[param_names[0]] = param1
            kwargs[param_names[1]] = param2
            noisy_img = noise_function(img, **kwargs)
            current_psnr = psnr(img, noisy_img)
            if (target_psnr - tol) <= current_psnr <= (target_psnr + tol):
                opt_params[(round(param1, 5), round(param2, 5))] = current_psnr
    return opt_params

def psnr(input, target, max_val=1.):
    mse = F.mse_loss(input, target)
    return 20 * torch.log10(max_val / torch.sqrt(mse))

img = np.load(f'./ground_truth/64/0.npy').squeeze()
img_torch = phantom_to_torch(img)
target_psnr_value = 27.6

# For salt and pepper noise
optimal_salt_pepper_params = find_noise_factor_for_psnr_2D(target_psnr_value, img_torch, add_salt_and_pepper_noise, ['salt_prob', 'pepper_prob'])

# Convert the dictionary to a Pandas DataFrame for better representation and export
df_salt_pepper = pd.DataFrame(list(optimal_salt_pepper_params.items()), columns=['(Salt_prob, Pepper_prob)', 'PSNR'])

# Print or save to CSV
print(df_salt_pepper)

# all but quantization noise looks good - not the right noise for denoising anyway

In [ ]:

img = np.load(f'./ground_truth/64/0.npy').squeeze()
img_torch = phantom_to_torch(img)

# add {type} noise to the image
noise_type = 'quantization'
noise_img = add_selected_noise(img_torch, noise_type=noise_type, levels=2.0)

# plot the images side by side
plt.figure()
plt.subplot(1, 2, 1)
plt.imshow(img, cmap='gray')
plt.title('Original')
plt.subplot(1, 2, 2)
plt.imshow(noise_img.squeeze(), cmap='gray')
plt.title(f'{noise_type} Noise')

# calculate the psnr
# these need to be numpy arrays
psnr = compare_psnr(img_torch.numpy(), noise_img.numpy())
print(f'PSNR: {psnr}')

# now create noise level and resolution subfolders in the sample folder

In [ ]:
# first resolution subfolders 64, 128, 256, 512
# next noise level subfolders 0.05, 0.09, .15, .20

# these are subfolders of the noise type folders, which are subfolders of the current directory
# so the path is ./noise_type/resolution/noise_level/phantom_phantomid_noisetype_noiselevel.npy
for key in NOISE_FUNCTIONS.keys():
    for res in [6, 7, 8, 9]:
        for noise_level in [0.05, 0.09, 0.15, 0.20]:
            # only create the folder if it doesn't exist
            os.makedirs(f'./{key}/resolution_{2**res}/noise_level_{noise_level}', exist_ok=True)

# now apply these noise levels to the ground truth

In [162]:
# apply to the first 64 res phantom and plot for sanity check first
# calculate the psnr
# these need to be numpy arrays

noise_level_table_20 = pd.read_csv('optimal_noise_factors_matching20g.csv')
noise_level_table_15 = pd.read_csv('optimal_noise_factors_matching15g.csv')
noise_level_table_9 = pd.read_csv('optimal_noise_factors_matching9g.csv')
noise_level_table_5 = pd.read_csv('optimal_noise_factors_matching5g.csv')

for key in NOISE_FUNCTIONS.keys():
    print(f'\n----------------------------------\nNoise Type: {key}\n')
    print(f"\t{'Resolution':<12}{'NF':<10}{'PSNR':<10}{'Target PSNR':<15}{'PSNR Error':<15}")
    for res in [6, 7, 8, 9]:
        for noise_level in [0.05, 0.09, 0.15, 0.20]:
            # load the phantom
            img = np.load(f'./ground_truth/{2**res}/0.npy').squeeze()
            img_torch = phantom_to_torch(img)
            
            if noise_level == 0.05:
                target_psnr = 27.6
            elif noise_level == 0.09:
                target_psnr = 22
            elif noise_level == 0.15:
                target_psnr = 18.2
            elif noise_level == 0.20:
                target_psnr = 15.7

            # Initialize an empty dictionary for noise parameters
            noise_kwargs = {}

            # now import the noise level that is actually used:
            # match key to Noise_Type and return Avg_Optimal_Factor
            # use if else to get right table
            if int(noise_level*10) == 20:
                noise_factor = noise_level_table_20.loc[noise_level_table_20['Noise_Type'] == key]['Avg_Optimal_Factor'].values[0]
            elif int(noise_level*10) == 15:
                noise_factor = noise_level_table_15.loc[noise_level_table_15['Noise_Type'] == key]['Avg_Optimal_Factor'].values[0]
            elif int(noise_level*10) == 9:
                noise_factor = noise_level_table_9.loc[noise_level_table_9['Noise_Type'] == key]['Avg_Optimal_Factor'].values[0]
            elif int(noise_level*10) == 5:
                noise_factor = noise_level_table_5.loc[noise_level_table_5['Noise_Type'] == key]['Avg_Optimal_Factor'].values[0]
            
            # Special cases for certain noise types
            if param_name:  # Check if a param_name exists
                noise_kwargs[param_name] = noise_level
            
            if key == 'salt_and_pepper':  # Special case for salt_and_pepper
                noise_kwargs = {'salt_prob': noise_level, 'pepper_prob': noise_level}
            
            if key == 'poisson':  # Special case for Poisson
                noise_kwargs = {}  # Poisson doesn't require any additional arguments
                
            # Add noise to the image
            noise_img = add_selected_noise(img_torch, noise_type=key, **noise_kwargs)
            
            # Calculate the psnr
            psnr = round(compare_psnr(img_torch.numpy(), noise_img.numpy()),3)

            psnr_error = round(abs(psnr-target_psnr), 3)
            print(f"\t{2**res:<12}{noise_factor:<10}{psnr:<10.4f}{target_psnr:<15.4f}{psnr_error:<15.3f}")


            # plot the images side by side
            # plt.figure()
            # # title plot with noise type, resolution, and noise level
            # plt.suptitle(f'{key.capitalize()} Noise -- Resolution: {2**res}x{2**res} \nNoise Level: {noise_level}')
            # plt.subplot(1, 2, 1)
            # plt.imshow(img, cmap='gray')
            # plt.title('Original')
            # plt.subplot(1, 2, 2)
            # plt.imshow(noise_img.squeeze(), cmap='gray')
            # plt.title(f'{key.capitalize()} Noise')
            # plt.show()
            # plt.close()


----------------------------------
Noise Type: gaussian

	Resolution  NF        PSNR      Target PSNR    PSNR Error     


NameError: name 'noise_factor' is not defined